In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.auto import tqdm
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch.nn as nn 
import torch 
import yaml 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score 
from torch.utils.data import Dataset, DataLoader

In [3]:
from laserscan import LaserScan

In [4]:
# Code adapted from SalsaNext @ Tiago Cortinhal 
class ClaSeKDataset(Dataset):

    def __init__(self, root,  # directory where data is
                 split,  # train or test
                 df, # pandas dataframe containing filename and label
                 label_dict,  # label dict: (e.g 10: "car")
                 sensor,  # sensor to parse scans from
                 max_points=150000,  # max number of points present in dataset
                 gt=True,
                 transform=False):
        # save deats
        self.root = os.path.join(root, split)
        self.label_dict = label_dict
        self.df = df
        self.sensor = sensor
        self.sensor_img_H = sensor["img_prop"]["height"]
        self.sensor_img_W = sensor["img_prop"]["width"]
        self.sensor_img_means = torch.tensor(sensor["img_means"],
                                             dtype=torch.float)
        self.sensor_img_stds = torch.tensor(sensor["img_stds"],
                                            dtype=torch.float)
        self.sensor_fov_up = sensor["fov_up"]
        self.sensor_fov_down = sensor["fov_down"]
        self.max_points = max_points
        self.gt = gt
        self.transform = transform

        self.nclasses = len(label_dict.keys())

        # make sure directory exists
        if os.path.isdir(self.root):
            print("Sequences folder exists! Using sequences from %s" % self.root)
        else:
            raise ValueError("Sequences folder doesn't exist! Exiting...")

        # make sure labels is a dict
        assert (isinstance(self.label_dict, dict))

    def __getitem__(self, index):
        # get item in tensor shape
        item = self.df.iloc[index]

        scan_file = os.path.join(self.root, item.FNAME)

        if self.gt:
            label = item.LABEL
        else:
            label = 0

        # open a semantic laserscan
        DA = False
        flip_sign = False
        rot = False
        drop_points = False
        if self.transform:
            if random.random() > 0.5:
                if random.random() > 0.5:
                    DA = True
                if random.random() > 0.5:
                    flip_sign = True
                if random.random() > 0.5:
                    rot = True
                drop_points = random.uniform(0, 0.5)

        scanner_class = LaserScan

        scan = scanner_class(project=True,
                                 H=self.sensor_img_H,
                                 W=self.sensor_img_W,
                                 fov_up=self.sensor_fov_up,
                                 fov_down=self.sensor_fov_down,
                                 DA=DA,
                                 rot=rot,
                                 flip_sign=flip_sign,
                                 drop_points=drop_points)


        scan.open_scan(scan_file)
        # make a tensor of the uncompressed data (with the max num points)
        unproj_n_points = scan.points.shape[0]
        if unproj_n_points < self.max_points:
            points_to_copy = unproj_n_points
        else:
            points_to_copy = self.max_points

        unproj_xyz = torch.full((self.max_points, 3), -1.0, dtype=torch.float)
        unproj_xyz[:points_to_copy] = torch.from_numpy(scan.points[: points_to_copy])
        unproj_range = torch.full([self.max_points], -1.0, dtype=torch.float)
        unproj_range[: points_to_copy] = torch.from_numpy(scan.unproj_range[: points_to_copy])
        unproj_remissions = torch.full([self.max_points], -1.0, dtype=torch.float)
        unproj_remissions[: points_to_copy] = torch.from_numpy(scan.remissions[: points_to_copy])

        # get points and labels
        proj_range = torch.from_numpy(scan.proj_range).clone()
        proj_xyz = torch.from_numpy(scan.proj_xyz).clone()
        proj_remission = torch.from_numpy(scan.proj_remission).clone()
        proj_mask = torch.from_numpy(scan.proj_mask)

        proj_x = torch.full([self.max_points], -1, dtype=torch.long)
        proj_x[: points_to_copy] = torch.from_numpy(scan.proj_x[: points_to_copy])
        proj_y = torch.full([self.max_points], -1, dtype=torch.long)
        proj_y[: points_to_copy] = torch.from_numpy(scan.proj_y[: points_to_copy])
        proj = torch.cat([proj_range.unsqueeze(0).clone(),
                          proj_xyz.clone().permute(2, 0, 1),
                          proj_remission.unsqueeze(0).clone()])
        proj = (proj - self.sensor_img_means[:, None, None]
                ) / self.sensor_img_stds[:, None, None]
        proj = proj * proj_mask.float()

        # return
        return proj, label, index

    def __len__(self):
        return self.df.shape[0]

    @staticmethod
    def map(label, mapdict):
        # put label from original values to xentropy
        # or vice-versa, depending on dictionary values
        # make learning map a lookup table
        maxkey = 0
        for key, data in mapdict.items():
            if isinstance(data, list):
                nel = len(data)
            else:
                nel = 1
            if key > maxkey:
                maxkey = key
        # +100 hack making lut bigger just in case there are unknown labels
        if nel > 1:
            lut = np.zeros((maxkey + 100, nel), dtype=np.int32)
        else:
            lut = np.zeros((maxkey + 100), dtype=np.int32)
        for key, data in mapdict.items():
            try:
                lut[key] = data
            except IndexError:
                print("Wrong key ", key)
        # do the mapping
        return lut[label]

In [5]:
# Code Adapted from SalsaNext @ Tiago Cortinhal 
class ResContextBlock(nn.Module):
    def __init__(self, in_filters, out_filters):
        super(ResContextBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_filters, out_filters, kernel_size=(1, 1), stride=1)
        self.act1 = nn.LeakyReLU()

        self.conv2 = nn.Conv2d(out_filters, out_filters, (3,3), padding=1)
        self.act2 = nn.LeakyReLU()
        #self.bn1 = nn.BatchNorm2d(out_filters)
        self.bn1 = nn.InstanceNorm2d(out_filters)

        self.conv3 = nn.Conv2d(out_filters, out_filters, (3,3),dilation=2, padding=2)
        self.act3 = nn.LeakyReLU()
        self.bn2 = nn.InstanceNorm2d(out_filters)


    def forward(self, x):
        shortcut = self.conv1(x)
        shortcut = self.act1(shortcut)
        resA = self.conv2(shortcut)
        resA = self.act2(resA)
        resA1 = self.bn1(resA)

        resA = self.conv3(resA1)
        resA = self.act3(resA)
        resA2 = self.bn2(resA)


        output = shortcut + resA2
        return output


class ResBlock(nn.Module):
    def __init__(self, in_filters, out_filters, dropout_rate, kernel_size=(3, 3), stride=1,
                 pooling=True, drop_out=True):
        super(ResBlock, self).__init__()
        self.pooling = pooling
        self.drop_out = drop_out
        self.conv1 = nn.Conv2d(in_filters, out_filters, kernel_size=(1, 1), stride=stride)
        self.act1 = nn.LeakyReLU()

        self.conv2 = nn.Conv2d(in_filters, out_filters, kernel_size=(3,3), padding=1)
        self.act2 = nn.LeakyReLU()
        #self.bn1 = nn.BatchNorm2d(out_filters)
        self.bn1 = nn.InstanceNorm2d(out_filters)

        self.conv3 = nn.Conv2d(out_filters, out_filters, kernel_size=(3,3),dilation=2, padding=2)
        self.act3 = nn.LeakyReLU()
        #self.bn2 = nn.BatchNorm2d(out_filters)
        self.bn2 = nn.InstanceNorm2d(out_filters)

        self.conv4 = nn.Conv2d(out_filters, out_filters, kernel_size=(2, 2), dilation=2, padding=1)
        self.act4 = nn.LeakyReLU()
        #self.bn3 = nn.BatchNorm2d(out_filters)
        self.bn3 = nn.InstanceNorm2d(out_filters)

        self.conv5 = nn.Conv2d(out_filters*3, out_filters, kernel_size=(1, 1))
        self.act5 = nn.LeakyReLU()
        #self.bn4 = nn.BatchNorm2d(out_filters)
        self.bn4 = nn.InstanceNorm2d(out_filters)

        if pooling:
            self.dropout = nn.Dropout2d(p=dropout_rate)
            #self.pool = nn.AvgPool2d(kernel_size=kernel_size, stride=2, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=kernel_size, stride=2, padding=1)
        else:
            self.dropout = nn.Dropout2d(p=dropout_rate)

    def forward(self, x):
        shortcut = self.conv1(x)
        shortcut = self.act1(shortcut)
        resA = self.conv2(x)
        resA = self.act2(resA)
        resA1 = self.bn1(resA)

        resA = self.conv3(resA1)
        resA = self.act3(resA)
        resA2 = self.bn2(resA)

        resA = self.conv4(resA2)
        resA = self.act4(resA)
        resA3 = self.bn3(resA)

        concat = torch.cat((resA1,resA2,resA3),dim=1)
        resA = self.conv5(concat)
        resA = self.act5(resA)
        resA = self.bn4(resA)


        resA = shortcut + resA

        if self.pooling:
            if self.drop_out:
                resB = self.dropout(resA)
            else:
                resB = resA
            resB = self.pool(resB)

            return resB, resA
        else:
            if self.drop_out:
                resB = self.dropout(resA)
            else:
                resB = resA
            return resB

class SalsaNext(nn.Module):
    def __init__(self, nclasses):
        super(SalsaNext, self).__init__()
        self.nclasses = nclasses

        self.downCntx = ResContextBlock(5, 32)

        self.downCntx2 = ResContextBlock(32, 32)
        self.downCntx3 = ResContextBlock(32, 32)

        self.resBlock1 = ResBlock(32, 2 * 32, 0.2, pooling=True, drop_out=False)
        self.resBlock2 = ResBlock(2 * 32, 2 * 2 * 32, 0.2, pooling=True)
        self.resBlock3 = ResBlock(2 * 2 * 32, 2 * 4 * 32, 0.2, pooling=True)
        self.resBlock4 = ResBlock(2 * 4 * 32, 2 * 4 * 32, 0.2, pooling=True)
        
        self.resBlock5 = ResBlock(2 * 4 * 32, 2 * 4 * 32, 0.2, pooling=True)
        self.resBlock6 = ResBlock(2 * 4 * 32, 2 * 4 * 32, 0.2, pooling=True)
        self.resBlock7 = ResBlock(2 * 4 * 32, 2 * 4 * 32, 0.2, pooling=True)

        self.resBlock8 = ResBlock(2 * 4 * 32, 2 * 4 * 32, 0.2, pooling=False)

        self.pool = nn.AdaptiveAvgPool2d(1)
        self.logits = nn.Conv2d(256, nclasses, kernel_size=(1, 1))

    def forward(self, x):
        downCntx = self.downCntx(x)
        downCntx = self.downCntx2(downCntx)
        downCntx = self.downCntx3(downCntx)

        down0c, down0b = self.resBlock1(downCntx)
        down1c, down1b = self.resBlock2(down0c)
        down2c, down2b = self.resBlock3(down1c)
        down3c, down3b = self.resBlock4(down2c)
        down4c, down4b = self.resBlock5(down3c)
        down5c, down5b = self.resBlock6(down4c)
        down6c, down6b = self.resBlock7(down5c)
        down7c = self.resBlock8(down6c)
        
        out = self.pool(down7c)

        logits = self.logits(out)
        
        return logits.squeeze()

In [6]:
LABEL_DICT = {0: "car", 1: "bicycle", 2: "motorcycle", 3: "truck", 4: "other_vehicle", 5: "person", 6: "motorcyclist",
              7: "road", 8: "parking", 9: "sidewalk", 10: "other_ground", 11: "building", 12: "fence", 13: "vegetation",
              14: "trunk", 15: "terrain", 16: "pole", 17: "traffic_sign"}
DATA_DIR = '../data/'
DATA_MEAN = np.array([0.43, 0.28, -0.67, 10.80]) # x, y, z, signal
DATA_STD = np.array([1.17, 1.40, 0.05, 0.97]) # x, y, z, signal
N_CLASSES = 18


In [7]:
conf_path = '../data/ds.yaml'
with open(conf_path, 'r') as stream:
    ds_conf = yaml.safe_load(stream)

sensor = ds_conf['dataset']['sensor']
max_points = ds_conf['dataset']['max_points']
train_file = os.path.join(DATA_DIR, 'train.csv')
test_file = os.path.join(DATA_DIR, 'test.csv')
test_df = pd.read_csv(test_file)
df = pd.read_csv(train_file)

In [8]:
train_df, val_df = train_test_split(df, stratify=df.LABEL.values, test_size=0.2)
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)

In [9]:
#Select device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda:0


In [10]:
batch_size = 8

train_ds = ClaSeKDataset(root=DATA_DIR,  # directory where data is
                 split='train',  # train or test
                 df=train_df, # pandas dataframe containing filename and label
                 label_dict=LABEL_DICT,  # label dict: (e.g 10: "car")
                 sensor=sensor,  # sensor to parse scans from
                 max_points=max_points,  # max number of points present in dataset
                 gt=True,
                 transform=False)
train_loader = DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=True)
val_ds = ClaSeKDataset(root=DATA_DIR,  # directory where data is
                 split='train',  # train or test
                 df=val_df, # pandas dataframe containing filename and label
                 label_dict=LABEL_DICT,  # label dict: (e.g 10: "car")
                 sensor=sensor,  # sensor to parse scans from
                 max_points=max_points,  # max number of points present in dataset
                 gt=True,
                 transform=False)
val_loader = DataLoader(dataset=val_ds, batch_size=batch_size, shuffle=False)

test_ds = ClaSeKDataset(root=DATA_DIR,  # directory where data is
                 split='test',  # train or test
                 df=test_df, # pandas dataframe containing filename and label
                 label_dict=LABEL_DICT,  # label dict: (e.g 10: "car")
                 sensor=sensor,  # sensor to parse scans from
                 max_points=max_points,  # max number of points present in dataset
                 gt=False,
                 transform=False)
test_loader = DataLoader(dataset=test_ds, batch_size=batch_size, shuffle=False)

Sequences folder exists! Using sequences from ../data/train
Sequences folder exists! Using sequences from ../data/train
Sequences folder exists! Using sequences from ../data/test


In [11]:
#get next item from train_loader
for i, (proj, label, index) in enumerate(train_loader):
    print(proj.shape)
    print(label.shape)
    print(index.shape)
    break

torch.Size([8, 5, 64, 2048])
torch.Size([8])
torch.Size([8])


In [12]:
#model = SalsaNext(nclasses=N_CLASSES)

In [13]:
#output = model(proj)

In [14]:
criterion = nn.CrossEntropyLoss().to(device)
# model = PointNet(15000, N_CLASSES).to(device)
model = SalsaNext(nclasses=N_CLASSES).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [15]:
def train_epoch(epoch, model, loader, criterion, optimizer, device):
    model.train()
    loss_list = list()
    pbar = tqdm(total=len(loader), desc=f'Epoch {epoch}')
    for i, (data, proj_labels, _) in enumerate(loader):
        data = data.float().to(device)
        label = proj_labels.long().to(device)
        optimizer.zero_grad()
        logit = model(data)
        loss = criterion(logit, label)
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
        pbar.update(1)
        pbar.set_postfix({'Loss': np.mean(loss_list)})
    pbar.close()
    return np.mean(loss_list)

def validate_epoch(epoch, model, loader, criterion, device):
    model.eval()
    loss_list = list()
    gts = list()
    preds = list()
    pbar = tqdm(total=len(loader), desc=f'Epoch {epoch}')
    with torch.no_grad():
        for i, (data, proj_labels,_) in enumerate(loader):
            data = data.float().to(device)
            label = proj_labels.long().to(device)
            logit = model(data)
            pred = torch.argmax(logit, dim=1)
            gts.extend(label.detach().cpu().numpy().tolist())
            preds.extend(pred.detach().cpu().numpy().tolist())
            loss = criterion(logit, label)
            loss_list.append(loss.item())
            pbar.update(1)
            pbar.set_postfix({'Loss': np.mean(loss_list)})
    pbar.close()
    gts = np.array(gts)
    preds = np.array(preds)
    acc = accuracy_score(y_true= gts, y_pred=preds)
    return np.mean(loss_list), acc


In [16]:
# Add the day and time of training to the model name
import datetime
now = datetime.datetime.now()
model_name = f'salsanext_{now.strftime("%Y-%m-%d_%H-%M-%S")}.pth'
model_base_path = '../models/'
model_path = os.path.join(model_base_path, model_name)
submission_path = os.path.join(model_base_path, f'submission_{model_name}.csv')
print(model_path)
print(submission_path)

../models/salsanext_2023-10-13_19-39-35.pth
../models/submission_salsanext_2023-10-13_19-39-35.pth.csv


In [17]:
epoch = 0
total_epoch = 10
train_losses = list()
best_val_acc = 0
best_val_loss = np.inf


Epoch 0: 100%|██████████| 27/27 [00:42<00:00,  1.59s/it, Loss=2.42]


Saving model at epoch: 0, val acc: 0.2132701421800948
Epoch: 0 Current val acc is 0.213 while the best val acc is 0.213


Epoch 1: 100%|██████████| 27/27 [00:44<00:00,  1.64s/it, Loss=1.87]


Saving model at epoch: 1, val acc: 0.33175355450236965
Epoch: 1 Current val acc is 0.332 while the best val acc is 0.332


Epoch 2: 100%|██████████| 27/27 [00:42<00:00,  1.59s/it, Loss=1.45]


...

Saving model at epoch: 7, val acc: 0.6161137440758294
Epoch: 7 Current val acc is 0.616 while the best val acc is 0.616


Epoch 8: 100%|██████████| 27/27 [00:42<00:00,  1.58s/it, Loss=0.999]


Epoch: 8 Current val acc is 0.588 while the best val acc is 0.616


Epoch 9: 100%|██████████| 27/27 [00:42<00:00,  1.58s/it, Loss=0.943]

Saving model at epoch: 9, val acc: 0.6398104265402843
Epoch: 9 Current val acc is 0.640 while the best val acc is 0.640


In [23]:
#Keep training
total_epoch = 35

In [24]:
while epoch < total_epoch:
    train_loss = train_epoch(epoch, model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate_epoch(epoch, model, val_loader, criterion, device)
    train_losses.append(train_loss)
    if best_val_acc < val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print(f'Saving model at epoch: {epoch}, val acc: {val_acc}')

    print('Epoch: %d Current val acc is %.3f while the best val acc is %.3f' %
          (epoch, val_acc, best_val_acc))
    epoch += 1

Epoch 10: 100%|██████████| 27/27 [00:39<00:00,  1.47s/it, Loss=0.943]


Epoch: 10 Current val acc is 0.640 while the best val acc is 0.640


Epoch 11: 100%|██████████| 27/27 [00:37<00:00,  1.39s/it, Loss=0.943]


Epoch: 11 Current val acc is 0.640 while the best val acc is 0.640


Epoch 12: 100%|██████████| 27/27 [00:37<00:00,  1.39s/it, Loss=0.943]


...

Epoch: 32 Current val acc is 0.640 while the best val acc is 0.640


Epoch 33: 100%|██████████| 27/27 [00:37<00:00,  1.39s/it, Loss=0.943]


Epoch: 33 Current val acc is 0.640 while the best val acc is 0.640


Epoch 34: 100%|██████████| 27/27 [00:37<00:00,  1.39s/it, Loss=0.943]

Epoch: 34 Current val acc is 0.640 while the best val acc is 0.640


In [25]:
def generate_submission_file(model, loader, device):
    model.eval()
    indices_list = list()
    preds = list()
    pbar = tqdm(total=len(loader), desc=f'Epoch {epoch}')
    with torch.no_grad():
        for i, (data, proj_labels, indices) in enumerate(
                loader):
            data = data.float().to(device)
            label = proj_labels.long().to(device)
            logit = model(data)
            pred = torch.argmax(logit, dim=1)
            indices_list.extend(indices.detach().cpu().numpy().tolist())
            preds.extend(pred.detach().cpu().numpy().tolist())
            pbar.update(1)
    pbar.close()
    sample = test_df.iloc[indices_list]
    sample['LABEL'] = preds
    return sample 

In [26]:
model = SalsaNext(nclasses=N_CLASSES).to(device)
model.load_state_dict(torch.load(model_path))
sample = generate_submission_file(model, test_loader, device)

Epoch 35:   0%|          | 0/150 [00:00<?, ?it/s]

Epoch 35: 100%|██████████| 150/150 [03:26<00:00,  1.38s/it]


In [27]:
sample.reset_index(drop=True, inplace=True)

In [28]:
sample

,FNAME,LABEL
0,06_122845.bin,15
1,08_925129.bin,14
2,08_976795.bin,9
3,06_317230.bin,0
4,08_386622.bin,0
...,...,...
1191,08_211179.bin,0
1192,08_672278.bin,11
1193,08_399783.bin,13
1194,05_205440.bin,15


In [29]:

sample.to_csv(submission_path, index=None)